### 1. 安装HuggingFace 并下载模型到本地

In [2]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 6.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.177.1-py2.py3-none-any.whl size=1166001 sha256=4d974d413f1f94e8f7623d5e37e63200ff49f761237cda0fd253383ddf1e6e92
  Stored in directory: /root/.cache/pip/wheels/4d/3f/29/a44088e382c815fd842f7d001b32aede28c9fdba7fd58ecf00
Successfully built sagemaker
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.175.0
    Uninstalling sagemaker-2.175.0:
      Successfully uninstalled sagemaker-2.175.0


-  可以把下面cell中的en全部切换成zh即可部署中文版

In [2]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./bge-zh-model")
local_model_path.mkdir(exist_ok=True)
model_name = "BAAI/bge-large-zh"
commit_hash = "1b543b301eb63dd32914b56d939db2a972df15d5"

In [3]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

'bge-zh-model/models--BAAI--bge-large-zh/snapshots/1b543b301eb63dd32914b56d939db2a972df15d5'

### 2. 把模型拷贝到S3为后续部署做准备

In [4]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [7]:
s3_model_prefix = "LLM-RAG/workshop/bge-zh-model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/bge_zh_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/bge_zh_deploy_code
model_snapshot_path: bge-zh-model/models--BAAI--bge-large-zh/snapshots/1b543b301eb63dd32914b56d939db2a972df15d5


In [8]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: bge-zh-model/models--BAAI--bge-large-zh/snapshots/1b543b301eb63dd32914b56d939db2a972df15d5/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-zh-model/.gitattributes
upload: bge-zh-model/models--BAAI--bge-large-zh/snapshots/1b543b301eb63dd32914b56d939db2a972df15d5/1_Pooling/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-zh-model/1_Pooling/config.json
upload: bge-zh-model/models--BAAI--bge-large-zh/snapshots/1b543b301eb63dd32914b56d939db2a972df15d5/config_sentence_transformers.json to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-zh-model/config_sentence_transformers.json
upload: bge-zh-model/models--BAAI--bge-large-zh/snapshots/1b543b301eb63dd32914b56d939db2a972df15d5/README.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-zh-model/README.md
upload: bge-zh-model/models--BAAI--bge-large-zh/snapshots/1b543b301eb63dd32914b56d939db2a972df15d5/config.json to s3://sagemaker-us-east-2-946277762357/LLM-RA

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [10]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [11]:
!mkdir -p bge_zh_deploy_code

In [12]:
%%writefile bge_zh_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
# from FlagEmbedding import FlagModel
from sentence_transformers import SentenceTransformer


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")

    # model =  FlagModel(model_location)
    model = SentenceTransformer(model_location)
    
    return model

model = None

def handle(inputs: Input):
    global model
    if not model:
        model = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = None
    inputs = data["inputs"]
    if isinstance(inputs, list):
        input_sentences = inputs
    else:
        input_sentences =  [inputs]
        
    is_query = data["is_query"]
    instruction = data["instruction"]
    logging.info(f"inputs: {input_sentences}")
    logging.info(f"is_query: {is_query}")
    logging.info(f"instruction: {instruction}")
    
    if is_query and instruction:
        input_sentences = [ instruction + sent for sent in input_sentences ]
        
    sentence_embeddings =  model.encode(input_sentences,normalize_embeddings=True)
        
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Writing bge_zh_deploy_code/model.py


In [13]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-zh-model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [14]:
%%writefile bge_zh_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge-zh-model/

Writing bge_zh_deploy_code/serving.properties


In [15]:
%%writefile bge_zh_deploy_code/requirements.txt
transformers==4.30.2
sentence-transformers

Writing bge_zh_deploy_code/requirements.txt


In [16]:
!rm s2e_model.tar.gz
!cd bge_zh_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz bge_zh_deploy_code

bge_zh_deploy_code/
bge_zh_deploy_code/model.py
bge_zh_deploy_code/requirements.txt
bge_zh_deploy_code/serving.properties


In [17]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/bge_zh_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [18]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("bge-large-zh") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bge-large-zh-2023-08-18-02-01-52-029
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/bge-large-zh-2023-08-18-02-01-52-029


In [22]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/bge-large-zh-2023-08-18-02-01-52-029-config',
 'ResponseMetadata': {'RequestId': '57d9d016-ed97-4d47-9d52-eef9c7aaa384',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '57d9d016-ed97-4d47-9d52-eef9c7aaa384',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'date': 'Fri, 18 Aug 2023 02:05:49 GMT'},
  'RetryAttempts': 0}}

In [23]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bge-large-zh-2023-08-18-02-01-52-029-endpoint


In [24]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/bge-large-zh-2023-08-18-02-01-52-029-endpoint
Status: InService


### 5. 设置autoscaling(optional)

In [ ]:
asg = boto3.client('application-autoscaling')

# Resource type is variant and the unique identifier is the resource ID.
resource_id=f"endpoint/{endpoint_name}/variant/variant1"

# scaling configuration
response = asg.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', 
    MinCapacity=1,
    MaxCapacity=4
)

In [ ]:
response = asg.put_scaling_policy(
    PolicyName=f'Request-ScalingPolicy-{endpoint_name}',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 10.0, # Threshold
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance',
        },
        'ScaleInCooldown': 300, # duration until scale in
        'ScaleOutCooldown': 60 # duration between scale out
    }
)

### 6. 模型测试

In [25]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
    }
    # instruction =  "Represent this sentence for searching relevant passages:"
    instruction = "为这个句子生成表示以用于检索相关文章："
    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "is_query": True,
                "instruction" : instruction
            }
        ),
        ContentType="application/json",
    )
    # 中文instruction => 为这个句子生成表示以用于检索相关文章：
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

In [27]:
prompts1 = ["MSK是什么？", "aws cleanroom是什么？"]

emb = get_vector_by_sm_endpoint(prompts1, smr_client, endpoint_name)
print(len(emb[0]))
print(emb[1])

1024
[0.02285151369869709, -0.015070997178554535, -0.013117046095430851, 0.00557536818087101, 0.021118616685271263, 0.024973174557089806, -0.017808184027671814, -0.02665492706000805, 0.025437485426664352, 0.01158227026462555, 0.0009396620444022119, 0.06347035616636276, -0.023801323026418686, -0.02833905629813671, -0.03059929795563221, -0.021495161578059196, 0.05410164222121239, 0.00881141796708107, 0.001953735714778304, -0.027573958039283752, 0.024827953428030014, 0.0064697908237576485, -0.011672352440655231, -0.015389867126941681, -0.016045548021793365, 0.0032304946798831224, 0.009333808906376362, -0.008296824991703033, 0.03669346496462822, -0.034027907997369766, -0.0313534289598465, -0.011205460876226425, 0.014368308708071709, 0.01182760950177908, -0.03035094030201435, -0.004441733006387949, -0.014640008099377155, 0.02478623390197754, 0.0019168019061908126, -0.037123315036296844, -0.01629898138344288, 0.00479609239846468, 0.013669213280081749, 0.015322056598961353, 0.0022918314207345

In [28]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [29]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [30]:
!aws sagemaker delete-model --model-name {model_name}